In [ ]:
import torch, math, matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

plt.ioff()

rule_number, time = 210, 2**18 # parameters

state = torch.zeros((7,5), dtype=torch.int64); state[3,2]=1
rule = torch.tensor(([int(d) for d in 8*'0'+str(bin(rule_number))[2:]])[:-9:-1])
population = [1]
center_column = [[1,0,0]]

for step in tqdm(range(1,time+1)):
    if step % 4 == 0:
        state = F.pad(state, (2,2,4,4), "constant", 0)
        height, width = state.size(dim=0), state.size(dim=1)
    config = 4*state + torch.roll(state, 1, 0) + torch.roll(state, -1, 0)
    config[0::2,0::2] += torch.roll(state, 1, 1)[0::2,0::2]
    config[1::2,0::2] += torch.roll(state, -1, 1)[1::2,0::2]
    config[0::2,1::2] += torch.roll(state, -1, 1)[0::2,1::2]
    config[1::2,1::2] += torch.roll(state, 1, 1)[1::2,1::2]
    state = rule.take(config)
    population.append(int(torch.sum(state)))
    center_column.append([int(state[int((height-1)/2)+x,int((width-1)/2)]) for x in (0,1,2)])
    if step % 512 == 0:
        f = open("_full/rule-"+str(rule_number)+"-state-"+str(step)+".txt", "w")
        f.write(str(state.tolist()).replace('[', '{').replace(']', '}'))
        f.close()
        f = open("_full/rule-"+str(rule_number)+"-population-"+str(step)+".txt", "w")
        f.write(str(population).replace('[', '{').replace(']', '}'))
        f.close()
        f = open("_full/rule-"+str(rule_number)+"-center_column-"+str(step)+".txt", "w")
        f.write(str(center_column).replace('[', '{').replace(']', '}'))
        f.close()
        plt.axis('off'); plt.imshow(state, aspect=1/math.sqrt(3), interpolation='antialiased')
        plt.savefig("_full/rule-"+str(rule_number)+"-plot-"+str(step)+".png",bbox_inches='tight',dpi=500)

  6%|▋         | 16390/262144 [36:21:11<545:05:05,  7.98s/it]   


KeyboardInterrupt: 